For AdaIN, I integrated the pretrained Magenta TensorFlow Hub model as the encoder–decoder backbone, since training AdaIN from scratch was outside scope given its complexity. My contribution was to extend and adapt the pretrained pipeline, adding multi-style blending, colour preservation, sharpness control, and foreground masking. These modifications transformed AdaIN into a flexible, interactive system and ensured it could be fairly compared with the scratch-built VGG19 and MobileNetV2 pipelines within the final 3-in-1 prototype.

In [1]:
# Install required libraries for deep learning operations
!pip install -q tensorflow tensorflow_hub gradio

In [3]:
# Import TensorFlow for deep learning operations
import tensorflow as tf
# TensorFlow Hub to load pretrained models such as AdaIN
import tensorflow_hub as hub
import numpy as np
# Gradio to build a simple user interface
import gradio as gr
from PIL import Image


In [11]:
# Function to load an uploaded image and prepare it for a model
def load_image_from_upload(image, target_size=(256, 256)):

    # Convert the uploaded image to RGB to ensure there are 3 channels
    image = image.convert("RGB").resize(target_size)
    # Turn the PIL image into a NumPy array and scale pixel values to [0,1]
    img = np.array(image).astype(np.float32) / 255.0
    # Add a batch dimension so the shape becomes [1, height, width, channels]
    img = tf.expand_dims(img, axis=0)
    return tf.convert_to_tensor(img, dtype=tf.float32)


In [12]:
# Function to apply style transfer on a content image with a given style image
def stylize_image(content_img, style_img):
    try:
        # If the content image or style image is a NumPy array, convert it back to a PIL image
        if isinstance(content_img, np.ndarray):
            content_img = Image.fromarray(content_img.astype("uint8"))
        if isinstance(style_img, np.ndarray):
            style_img = Image.fromarray(style_img.astype("uint8"))
        content_tensor = load_image_from_upload(content_img)

        # Preprocess the style image into a model-ready tensor
        style_tensor = load_image_from_upload(style_img)
        # Run the style transfer model on the content and style tensors
        stylized_image = model(content_tensor, style_tensor)[0]
        # Remove the batch dimension and convert to NumPy array
        output = tf.squeeze(stylized_image).numpy()
        # Rescale pixel values
        output = np.clip(output * 255, 0, 255).astype('uint8')
        # Convert the NumPy array back into a PIL image for display/output
        return Image.fromarray(output)

    except Exception as e:
        print(" ERROR:", e)
        return Image.new('RGB', (256, 256), color='red')


In [13]:
# Load the pretrained AdaIN style transfer model from TensorFlow Hub
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

# Function to blend multiple styles using AdaIN with weighted alphas
def blend_styles_with_adain(content_img, style_imgs, alphas):
    # Preprocess style image to tensors
    style_tensors = [
        load_image_from_upload(Image.fromarray(img.astype("uint8")) if isinstance(img, np.ndarray) else img)
        for img in style_imgs
    ]

    # Normalize style weights
    alphas = np.array(alphas)
    alphas = alphas / alphas.sum()

    # Start with a zero tensor and add each style weighted by its alpha
    blended_style = tf.zeros_like(style_tensors[0])
    for i in range(len(style_tensors)):
        blended_style += style_tensors[i] * alphas[i]

    # Preprocess the content image (NumPy → PIL if needed, then to tensor)
    content_tensor = load_image_from_upload(
        Image.fromarray(content_img.astype("uint8")) if isinstance(content_img, np.ndarray) else content_img
    )

    # Run style transfer model with the blended style tensor
    stylized = model(content_tensor, blended_style)[0]
    output = tf.squeeze(stylized).numpy()
    output = np.clip(output * 255, 0, 255).astype('uint8')

    # Convert NumPy array back into a PIL image
    return Image.fromarray(output)


In [14]:
# Install the OpenCV library (cv2) for image processing functions
!pip install opencv-python


In [15]:
# Import the OpenCV library
import cv2
# Print the installed OpenCV version to confirm successful installation
print(cv2.__version__)


4.12.0


In [17]:
import gradio as gr
from PIL import Image, ImageFilter
import numpy as np
import cv2

# Only applies style to those black pixels
def make_black_mask_from_np(pil_img, threshold=30):
    np_img = np.array(pil_img)
    lower = np.array([0, 0, 0])
    upper = np.array([threshold, threshold, threshold])
    mask = cv2.inRange(np_img, lower, upper)
    return mask // 255  # binary mask 0/1


# Apply style only to foreground if white background is
def apply_fg_aware_style(content_pil, stylized_pil, fg_checkbox_enabled=True):
    if fg_checkbox_enabled:
        mask = make_black_mask_from_np(content_pil, threshold=30)
        mask_3ch = np.stack([mask]*3, axis=-1)
        content_np = np.array(content_pil.convert("RGB"))
        stylized_np = np.array(stylized_pil.convert("RGB"))
        blended = np.where(mask_3ch == 1, stylized_np, content_np)
        return Image.fromarray(blended.astype(np.uint8))
    else:
        return stylized_pil


# Your existing style blending logic will be called here
def process(content_img, style1, style2, style3, style4, style5,
            s1_strength, s2_strength, s3_strength, s4_strength, s5_strength,
            sharpness_value, preserve_colour, fg_checkbox):

    style_imgs = [style1, style2, style3, style4, style5]
    style_strengths = [s1_strength, s2_strength, s3_strength, s4_strength, s5_strength]

    valid_imgs = []
    valid_weights = []

    for img, w in zip(style_imgs, style_strengths):
        if img is not None and w > 0:
            valid_imgs.append(img)
            valid_weights.append(w)

    if len(valid_imgs) == 0:
        raise gr.Error("At least one style image with non-zero strength is required.")

    total = sum(valid_weights)
    valid_weights = [w / total for w in valid_weights]

    content_img = Image.fromarray(content_img) if isinstance(content_img, np.ndarray) else content_img
    content_img_resized = content_img.resize((256, 256))

    #Call blending function for AdaIN
    output_pil = blend_styles_with_adain(content_img_resized, valid_imgs, valid_weights)

    # Preserve original colors if requested
    if preserve_colour:
        try:
            stylized_np = np.array(output_pil.convert("RGB")).astype(np.uint8)
            content_np = np.array(content_img_resized.convert("RGB")).astype(np.uint8)
            stylized_yuv = cv2.cvtColor(stylized_np, cv2.COLOR_RGB2YUV)
            content_yuv = cv2.cvtColor(content_np, cv2.COLOR_RGB2YUV)
            combined_yuv = content_yuv.copy()
            combined_yuv[..., 0] = stylized_yuv[..., 0]
            final_rgb = cv2.cvtColor(combined_yuv, cv2.COLOR_YUV2RGB)
            output_pil = Image.fromarray(final_rgb)
        except Exception as e:
            raise gr.Error(f"Colour preservation failed: {e}")

    # Apply sharpness filter
    output_pil = output_pil.filter(ImageFilter.UnsharpMask(radius=2, percent=int(sharpness_value * 200)))

    # Apply foreground-aware blending
    output_pil = apply_fg_aware_style(content_img_resized, output_pil, fg_checkbox)

    return output_pil

#  GRADIO UI definition
with gr.Blocks() as demo:
    # App title and description
    gr.Markdown("##  AdaIN Multi-Style Transfer App")
    gr.Markdown("Upload one content image and up to five style images. Control style influence, sharpness, and optional foreground-aware styling.")
    # Content image input
    with gr.Row():
        content_input = gr.Image(label=" Content Image", image_mode="RGB", height=256, width=256)
    # Style image inputs
    with gr.Row():
        style_input1 = gr.Image(label=" Style Image 1 (Required)", image_mode="RGB", height=256, width=256)
        style_input2 = gr.Image(label=" Style Image 2 (Optional)", image_mode="RGB", height=256, width=256)
        style_input3 = gr.Image(label=" Style Image 3 (Optional)", image_mode="RGB", height=256, width=256)
        style_input4 = gr.Image(label=" Style Image 4 (Optional)", image_mode="RGB", height=256, width=256)
        style_input5 = gr.Image(label=" Style Image 5 (Optional)", image_mode="RGB", height=256, width=256)
    # Sliders for style blending strengths
    with gr.Row():
        slider1 = gr.Slider(minimum=0.0, maximum=1.0, value=1.0, label="Style 1 Strength")
        slider2 = gr.Slider(minimum=0.0, maximum=1.0, value=0.0, label="Style 2 Strength")
        slider3 = gr.Slider(minimum=0.0, maximum=1.0, value=0.0, label="Style 3 Strength")
        slider4 = gr.Slider(minimum=0.0, maximum=1.0, value=0.0, label="Style 4 Strength")
        slider5 = gr.Slider(minimum=0.0, maximum=1.0, value=0.0, label="Style 5 Strength")
    #  Sliders and checkboxes for hyperparameters
    with gr.Row():
        sharpness_slider = gr.Slider(minimum=0.0, maximum=1.0, value=0.5, label=" Style Sharpness")

    with gr.Row():
        preserve_colour = gr.Checkbox(label="Apply Colour Preservation", value=False)
        fg_checkbox = gr.Checkbox(label="Apply Foreground-Aware Styling", value=False)
    # Stylise button and output image
    run_button = gr.Button(" Stylise")
    output_image = gr.Image(label=" Stylised Output", image_mode="RGB", height=256, width=256)

    # Connect button to callback
    run_button.click(
        fn=process,
        inputs=[
            content_input,
            style_input1, style_input2, style_input3, style_input4, style_input5,
            slider1, slider2, slider3, slider4, slider5,
            sharpness_slider,
            preserve_colour,
            fg_checkbox
        ],
        outputs=output_image
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1ee812080ca2b21e6b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
